In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
# 读取Excel文件
df = pd.read_excel(r'D:software\lily-pad\sanjiaosanyuanzhu\KC=5.1.xlsx')
# 初始参数猜测
popt0=[ 0,  1,  0,  0, 0,  0.9,1]
popt0=[ 0,  1,  0,  0, 0,  0.9,0,0,1,1]
popt0=[ 1,  1,  1,  1, 1,  1,1,1]#KC=1,3
popt0=[ 0,  0,  0,  0, 0,  0,0,0]#KC=5,7
#popt0=[-2.9033516298929785, 6.868745182212709, 8.555376028763773, -13.00723753228518, -3.2663711649382945, 9.77092294470448, 4.487283981397603]
# 定义非线性模型函数
def nonlinear_func(x, a, b, c, d,e,f,g,h):
    x1 = x[:, 0]  # 提取x1数据
    x2 = x[:, 1]  # 提取x2数据
    return a*( (x1 +1)/(x1)) * ((x2 + f) / (x2 + 1))* ((x1 + 1)/(x1)) * ((x2 + f) / (x2 + 1)) * ((x2 +g*x1*x1+ b*x1 + c) / (x2 +h*x1*x1+ d*x1 + e))

# 提取数据
x1_data = df.iloc[:, 5].values
x2_data = df.iloc[:, 6].values
y_data = df.iloc[:, 0].values/3#Cd之前的数据没有取平均

# 创建参数数组
params_array = np.column_stack((x1_data, x2_data))
# 参数的边界限制
bounds = ([-10, -10,-10 ,-10, -10,-10,-10,-10], [10, 10, 10, 10, 10, 10,10,10])
# 使用curve_fit进行非线性回归拟合
popt, pcov = curve_fit(nonlinear_func, params_array, y_data, p0=popt0,maxfev=1000000000)
# 或者使用 least_squares 函数
from scipy.optimize import least_squares

initial_params = np.array(popt0)
#result = least_squares(nonlinear_func, initial_params, args=(params_array, y_data))

#popt_least_squares = result.x
#pcov_least_squares = result.covar
# 使用拟合参数进行预测
y_pred = nonlinear_func(params_array, *popt)

# 计算R²分数
r2 = r2_score(y_data, y_pred)
mse=mean_squared_error(y_data, y_pred)
mae=mean_absolute_error(y_data, y_pred)
# 打印拟合参数和R²值
print("拟合参数：a={}, b={}, c={}, d={},e={},f={},g={},h={}".format(*popt))
print("R²值：", r2)
print("MAE值：", mae)
print("MSE值：", mse)

拟合参数：a=0.019532201326975022, b=110.68560605844222, c=-19.208938290038514, d=0.09658374716922337,e=-0.2667328194207011,f=-0.19217506596047024,g=-15.157928010574935,h=-0.02559318611799217
R²值： -5.922985029581201
MAE值： 0.1808651690645035
MSE值： 0.08704917156750903


In [10]:

from scipy.optimize import curve_fit, least_squares

# 定义残差函数，用于 least_squares
def residuals(params, x, y):
    # 计算残差：模型预测 - 实际值
    a=params[0]
    b = params[1]
    c = params[2]
    d=params[3]
    e=params[4]
    f=params[5]
    g=params[6]
    h=params[7]
    popt1=[a,b,c,d,e,f,g,h]
    #y1=nonlinear_func(x, a, b, c, d,e,f,g)
    popt1, pcov1 = curve_fit(nonlinear_func, x, y, p0=popt1,maxfev=1000000)
    y2 = nonlinear_func(x, *popt1)
    return  mean_squared_error(y, y2)
# 使用 least_squares 函数进行拟合
initial_params = np.array(popt)
result = least_squares(residuals, initial_params, args=(params_array, y_data))

popt_least_squares = result.x
#pcov_least_squares = result.covar
popt1, pcov1 = curve_fit(nonlinear_func, params_array, y_data, p0=popt_least_squares,maxfev=1000000)
y_pred_least_squares = nonlinear_func(params_array, *popt1)

# 计算 R² 分数、MSE 和 MAE
r2 = r2_score(y_data, y_pred)
mse = mean_squared_error(y_data, y_pred)
mae = mean_absolute_error(y_data, y_pred)

r2_least_squares = r2_score(y_data, y_pred_least_squares)
mse_least_squares = mean_squared_error(y_data, y_pred_least_squares)
mae_least_squares = mean_absolute_error(y_data, y_pred_least_squares)

# 打印拟合参数和评估指标
print("拟合参数（curve_fit）：", popt)
print("R²值（curve_fit）：", r2)
print("MAE值（curve_fit）：", mae)
print("MSE值（curve_fit）：", mse)

print("拟合参数（least_squares）：", popt1)
print("R²值（least_squares）：", r2_least_squares)
print("MAE值（least_squares）：", mae_least_squares)
print("MSE值（least_squares）：", mse_least_squares)

TypeError: nonlinear_func() missing 1 required positional argument: 'h'

import random
from deap import base, creator, tools, algorithms
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

# 读取数据和定义非线性模型函数
# 假设 df, nonlinear_func, params_array, x1_data, x2_data, y_data 已经定义并初始化
popt0=[ 3.99752863,  0.10000416, -0.09680028,  0.69982798, -0.30078727,  1.59933251,0.20094101]
# 定义评估函数，即适应度函数
def evalModel(individual):
    a, b, c, d, e, f, g = individual
    #y_pred = nonlinear_func(params_array, a, b, c, d, e, f, g)
    popt1=[a,b,c,d,e,f,g]
    #y1=nonlinear_func(x, a, b, c, d,e,f,g)
    popt1, pcov1 = curve_fit(nonlinear_func, params_array, y_data, p0=popt1,maxfev=10000)
    y2 = nonlinear_func(params_array, *popt1)
    R2 = r2_score(y_data, y2)
    # 将计算得到的 mse 转换为适应度，并赋值给个体的 fitness 属性
    return R2,

# 创建适应度和个体类型
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 适应度最小化
creator.create("Individual", list, fitness=creator.FitnessMin)

# 初始化工具箱并注册必要的组件
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 7)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalModel)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# 遗传算法参数
POP_SIZE = 300
CX_PROB = 0.7
MUT_PROB = 0.2
NGEN = 50

# 初始化种群
pop = toolbox.population(n=POP_SIZE)

for ind in pop:
    ind[:] = popt0
# 评估初始种群
for ind in pop:
    fitness = toolbox.evaluate(ind)
    ind.fitness.values = fitness  # 将适应度值赋值给个体的 fitness 属性

# 遗传算法主循环
for gen in range(NGEN):
    print(f"Generation {gen}: {min(ind.fitness.values[0] for ind in pop if not ind.fitness.values[0] == np.inf)}")
    
    # 选择
    offspring = algorithms.varAnd(pop, toolbox, CX_PROB, MUT_PROB)
    
    # 评估
    for ind in offspring:
        fitness = toolbox.evaluate(ind)
        ind.fitness.values = fitness
    
    # 替换
    pop = toolbox.select(offspring, k=len(pop))

# 找到最优解
best_ind = tools.selBest(pop, 1)
best_fitness = best_ind[0].fitness.values[0]
print(f"Best individual is {best_ind[0]}, with fitness {best_fitness}")
print(f"Optimal parameters: a={best_ind[0][0]}, b={best_ind[0][1]}, c={best_ind[0][2]}, d={best_ind[0][3]}, e={best_ind[0][4]}, f={best_ind[0][5]}, g={best_ind[0][6]}")

In [ ]:
KC=3:
    
    Cm=拟合参数（curve_fit）： [ 2.2752216   6.49624501  8.93019631  4.36538839 -0.39634102  0.43941601
  0.76773292]
R²值（curve_fit）： 0.9021212636617872
MAE值（curve_fit）： 0.03676863622996852
MSE值（curve_fit）： 0.003390915176320596
拟合参数（least_squares）： [ 2.27525841  6.49606704  8.92993876  4.36534937 -0.39634247  0.43940618
  0.76771665]
R²值（least_squares）： 0.9021212636791467
MAE值（least_squares）： 0.03676882731863883
MSE值（least_squares）： 0.003390915175719194

